# 1. Install and import libraries

In [ ]:
!pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
!pip install -qq accelerate==0.26.1
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine 

In [ ]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# 2. Connect to Drive, Huggingface and load Poem dataset




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATASET_PATH = '/content/drive/MyDrive/poem_dataset.csv'
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Bạn xấu như chiếc bóng”,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,3,“Chơi thân không có nghĩa”,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Có thể buồn chút ít”,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
95,95,Ai về khuya,Chợ trưa ai đợi mẹ\nBến khuya ai đợi đò\nSông ...,NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
96,96,Ám ảnh,Người ấy đến rất say\ntừ cao trời bay xuống\nm...,Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
97,97,Ám ảnh,Tôi giật mình đánh rơi\nCon mèo con xuống giến...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/T%C3%B4-H%C3%A0/%C3%81...
98,98,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...


In [ ]:
with open('/content/drive/MyDrive/hf_token 2.txt', 'r') as file:
    hf_token = file.read().strip()




In [ ]:
from huggingface_hub import HfApi

# Replace 'your_token_here' with your actual Hugging Face API token
api = HfApi()
whoami = api.whoami(token=hf_token)
print(whoami)


{'type': 'user', 'id': '66b7b25902fd8eb58bc5a8c0', 'name': 'NhatNguyen2101', 'fullname': 'Nhat Minh Nguyen', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/noauth/MyMiGIJ8yHZHnyULc_eMi.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'Poem_gen project', 'role': 'fineGrained', 'createdAt': '2024-08-15T18:22:49.927Z', 'fineGrained': {'canReadGatedRepos': True, 'global': ['inference.serverless.write', 'discussion.write', 'post.write'], 'scoped': [{'entity': {'_id': '66b7b25902fd8eb58bc5a8c0', 'type': 'user', 'name': 'NhatNguyen2101'}, 'permissions': ['repo.content.read', 'repo.write', 'inference.endpoints.infer.write', 'inference.endpoints.write', 'user.webhooks.read', 'user.webhooks.write', 'collection.read', 'collection.write', 'discussion.write']}]}}}}


In [ ]:
import os
os.environ["HF_HOME"] = hf_token
# print(os.getenv("HF_HOME"))  # Check if the token is set

In [ ]:
def split_content(content):
  samples = []

  poem_parts = content.split('\n\n')
  for poem_part in poem_parts:
    lines = poem_part.split('\n')
    if len(lines) == 4:
      samples.append(lines)

  return samples

df['content'] = df['content'].apply(lambda x: split_content(x))
df.head()

,Unnamed: 0,title,content,source,url
0,0,“Bạn xấu như chiếc bóng”,"[[Bạn xấu như chiếc bóng, Cứ bám riết theo anh...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Cái làm ta hạnh phúc”,"[[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhi...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,“Chiều vừa xốp trên tay”,"[[Chiều vừa xốp trên tay, Chợt nghe thoáng ong...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,3,“Chơi thân không có nghĩa”,"[[Chơi thân không có nghĩa, Không cãi nhau bao...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Có thể buồn chút ít”,"[[Có thể buồn chút ít, Một mình, không người y...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


In [ ]:
print (df['content'][0])

[['Bạn xấu như chiếc bóng', 'Cứ bám riết theo anh', 'Khi anh sáng, rực rỡ', 'Như mặt trời long lanh'], ['Nhưng họ sẽ biến mất', 'Khi trời phủ mây đen', 'Tức là khi anh đói', 'Trong túi không có tiền']]


In [ ]:
df_exploded = df.explode('content')
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=['content'])
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Bạn xấu như chiếc bóng”,"[Bạn xấu như chiếc bóng, Cứ bám riết theo anh,...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Bạn xấu như chiếc bóng”,"[Nhưng họ sẽ biến mất, Khi trời phủ mây đen, T...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Cái làm ta hạnh phúc”,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,1,“Cái làm ta hạnh phúc”,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,2,“Chiều vừa xốp trên tay”,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
...,...,...,...,...,...
228,98,Ám ảnh sông xưa,"[Ôi, con sóng chết khô,, vật vờ trong bùn quán...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
229,99,Áng dương không biết sầu,"[Áng dương không biết sầu, Nằm mãi ở trên cao,...","Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
230,99,Áng dương không biết sầu,"[Em ơi sao tàn nhẫn, Nỡ xa rời vòng tay, Một t...","Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
231,99,Áng dương không biết sầu,"[Đợi em những đêm dài, Không một lời đối đáp, ...","Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...


In [ ]:
df_exploded['content'] = df_exploded['content'].apply(lambda x: '\n'.join(x))
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Bạn xấu như chiếc bóng”,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Bạn xấu như chiếc bóng”,Nhưng họ sẽ biến mất\nKhi trời phủ mây đen\nTứ...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,1,“Cái làm ta hạnh phúc”,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,2,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
...,...,...,...,...,...
228,98,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
229,99,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
230,99,Áng dương không biết sầu,Em ơi sao tàn nhẫn\nNỡ xa rời vòng tay\nMột th...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
231,99,Áng dương không biết sầu,Đợi em những đêm dài\nKhông một lời đối đáp\nC...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...


In [ ]:
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset

Dataset({
    features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
    num_rows: 212
})

In [ ]:
TEST_SIZE = 0.1
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 190
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 22
    })
})

# 3. Preprocess dataset

In [ ]:
MODEL_NAME = 'danghuy1999/gpt2-viwiki'

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/773k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/431k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 100

def preprocess_function(row):
    return tokenizer(
        row['content'],
        max_length=MAX_SEQ_LEN,
        padding='max_length',
        truncation=True
    )

tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=poem_dataset['train'].column_names,
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/190 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/22 [00:00<?, ? examples/s]

In [ ]:
tokenizer('xin chào các bạn', max_length=10, padding='max_length', truncation=True)

{'input_ids': [6504, 3939, 331, 1486, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

In [ ]:
tokenized_poem_dataset['train'][0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 4. Model training

In [ ]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='gpt2_viet_poem_generation',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=30,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True,
    hub_token=hf_token
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset['train'],
    eval_dataset=tokenized_poem_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

Step,Training Loss
500,5.416700


TrainOutput(global_step=720, training_loss=5.083766598171658, metrics={'train_runtime': 520.315, 'train_samples_per_second': 10.955, 'train_steps_per_second': 1.384, 'total_flos': 290891520000000.0, 'train_loss': 5.083766598171658, 'epoch': 30.0})

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

events.out.tfevents.1723747311.239eb1825e01.150.0:   0%|          | 0.00/5.12k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/NhatNguyen2101/gpt2_viet_poem_generation/commit/c3aac359fa11f7ab238c4471a747b16eda518e7a', commit_message='Training complete', commit_description='', oid='c3aac359fa11f7ab238c4471a747b16eda518e7a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

prompt = 'Lại là một ngày mới\n'
generator = pipeline('text-generation', model='thangduong0509/gpt2_viet_poem_generation')
results = generator(
    prompt,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = results[0]['generated_text']
print()
for line in results.split('\n'):
    print(line)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/974k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/431k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Lại là một ngày mới
Một cách xa hơn trời
Làm về nước, xanh lá cây
Là người có thể sống dưới!
Đời gió mạnh? Một mùa?
Những cơn giông bão trong môi! Đó là những cơn mưa lớn
Chây vào đêm? Ngày


In [ ]:
prompt = 'Học học nữa học mãi\n'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = results[0]
print()
for line in results.split('\n'):
    print(line)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Học học nữa học mãi
Như là nó là do anh
Trong đời mình còn nhỏ
Do khó khăn cần ở?..."
Khi ấy, tuyệt vời hơn
Và ai tôi chẳng thương. Sự biến đổi ý?
Có thể nào trong đời... Và công sự!
